# Package 

In [22]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
import warnings
warnings.simplefilter('ignore')

import pickle
import gc
import re
import polars as pl
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm.auto import tqdm
import polars as pl
from utils import *
from src.eval import model_eval
from src.config import raw_data_session_id_dir, candidate_dir, model_for_eval

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# df_prod = pd.read_csv('data/products_train.csv')
# df_prod

# Config 

In [3]:
# ! ls ../model_training/

In [24]:
debug = False


# model_version = 'popular_v1'

topn = 100
if debug:
    n_rows = 1000
else:
    n_rows = None
# debug_session_num = 100
train_data_dir = '.'
test_data_dir = '.'
task = 'task1'

# model_dir = f'../model_training/{model_version}/'

# target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# if model_for_eval:
# model_file = os.path.join(model_dir, f'nic_{model_for_eval}_for_eval.parquet')
# submit_file = os.path.join('../data/sub_files/', f'submission_{task}_nic_{model_for_eval}_for_eval.parquet')

In [25]:
# candidate_file.format(
#     task='task1'
#     , data_type='train'
#     , model_version=model_version
#     , model_for_eval=model_for_eval
#     , topn=topn
# )

# Data 

In [26]:
train_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=n_rows).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=n_rows).with_columns(pl.col('prev_items').apply(str2list))
product_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 
                                          'products_train.parquet'))


In [27]:
product_pl.head().collect()

id,locale,title,price,brand,color,size,model,material,author,desc,unique_id
str,str,str,f64,str,str,str,str,str,str,str,i64
"""B005ZSSN10""","""DE""","""RED DRAGON Amb…",30.95,"""RED DRAGON""",null,null,"""RDD0089""",null,null,"""Amberjacks Ste…",0
"""B08PRYN6LD""","""DE""","""Simply Keto Lo…",17.9,"""Simply Keto""",null,"""750 g (1er Pac…",null,null,null,"""🌱 NATÜRLICHE S…",1
"""B09MBZJ48V""","""DE""","""Sennheiser 508…",68.89,"""Sennheiser""","""Multi-Colour""","""One size""","""508377""","""Kunstleder""",null,"""3.5 MM BUCHSE …",2
"""B08ZN6F26S""","""DE""","""AmyBenton Auto…",18.99,"""Amy & Benton""","""Animal Car""",null,"""2008B""","""aufziehauto 1 …",null,"""【Auto aufziehb…",3
"""B094DGRV7D""","""DE""","""PLAYMOBIL - 70…",7.17,"""PLAYMOBIL""","""Nicht Zutreffe…","""OneSize""","""70522""","""Polypropylen""",null,"""Inhalt: 1 Stüc…",4


# Session features 

In [28]:
cols_to_keep = ['session_id', 'prev_items', 'locale']
session_pl = pl.concat([
    train_pl.select(cols_to_keep),
    eval_pl.select(cols_to_keep), 
    test_pl.select(cols_to_keep)], how='vertical')
        

In [29]:
session_len = session_pl.select('session_id').collect().shape[0]
session_len

3589687

In [30]:
product_cols = ['id', 'locale', 'price', 'brand', 'color', 'size']
session_pl = (
    session_pl.explode('prev_items')
        .join(product_pl.select(product_cols), 
              left_on=['prev_items', 'locale'], 
             right_on=['id', 'locale']
             )
        .groupby(['session_id', 'locale'])
        .agg(
            pl.all()
        )
        .with_columns(
            pl.col('price').arr.mean().alias('mean_price')
            , pl.col('price').arr.get(-1).alias('last_price')
            , pl.col('price').arr.min().alias('min_price')
            , pl.col('price').arr.max().alias('max_price')
        )
)

In [31]:
session_pl = session_pl.collect()

In [32]:
assert session_pl.shape[0] == session_len

In [33]:
session_pl.write_parquet('../data/session_item_features/session_features.parquet')

# Top200 for fallback logics

In [11]:
train_data = train_pl.with_columns(
    pl.col('prev_items').arr.concat(pl.col('next_item')) 
)
if not model_for_eval:
    eval_data = eval_pl.with_columns(
    pl.col('prev_items').arr.concat(pl.col('next_item')) 
)
else:
    eval_data = eval_pl
test_data = test_pl
cols_to_keep = ['prev_items', 'locale']
popular_pl = pl.concat([train_data.select(cols_to_keep), eval_data.select(cols_to_keep), test_data.select(cols_to_keep)], how='vertical')
        

In [12]:
# next_item_df[cols].info()

In [13]:
popular_pl.head().collect()

prev_items,locale
list[str],str
"[""B09W9FND7K"", ""B09JSPLN1M"", ""B09M7GY217""]","""DE"""
"[""B076THCGSG"", ""B007MO8IME"", … ""B001B4THSA""]","""DE"""
"[""B0B1LGXWDS"", ""B00AZYORS2"", … ""B0767DTG2Q""]","""DE"""
"[""B0749V8TC7"", ""B0749W93VC"", … ""B0749TX4YS""]","""DE"""
"[""B09SMK3R8H"", ""B01N4ND0F9"", ""B08YNZT93Z""]","""DE"""


In [14]:
topn = 200
locale_popular_pl = (
    popular_pl
        .explode('prev_items')
        .groupby(['locale', 'prev_items'])
        .agg(
            pl.count()
        )
        .with_columns(
            pl.col('count').rank(method='ordinal', descending=True).over('locale').alias('rank')
        )
        .filter(pl.col('rank')<=topn)
        .with_columns(
            pl.col('count').max().over('locale').alias('max_count')
            , pl.col('count').min().over('locale').alias('min_count')
        )
        .with_columns(
            ((pl.col('count')-pl.col('min_count'))/(pl.col('max_count')-pl.col('min_count'))).alias('weight')
        )
        .sort('locale', 'rank')
        .select(
            'locale'
            , 'prev_items'
            , 'weight'
        )
        .groupby('locale')
        .agg(
            pl.col('weight').alias('locale_popular_weight')
            , pl.col('prev_items').alias('locale_popular_rec')
        )
        # .count()#.head(3).collect())
        # .collect()
)

In [15]:
locale_popular_pl.collect()

locale,locale_popular_weight,locale_popular_rec
str,list[f64],list[str]
"""UK""","[1.0, 0.911864, … 0.0]","[""B08CN3G4N9"", ""B07N8QY3YH"", … ""B0765ZQJYJ""]"
"""JP""","[1.0, 0.668449, … 0.0]","[""B0BD5MFPMF"", ""B0BD88WWQ8"", … ""B07PNGSC5S""]"
"""DE""","[1.0, 0.963384, … 0.0]","[""B0BDML9477"", ""B08GWS298V"", … ""B01GGKYLW0""]"


# Model Eval 

In [26]:
def get_next_items(x):
    prev_items = x['prev_items']
    local_rec = x['locale_popular_rec']
    final = [ele for ele in local_rec if ele not in prev_items]
    return final


eval_res = (
    eval_pl
        .join(locale_popular_pl, how='left', on='locale')
        .with_columns(
            pl.col('locale_popular_rec')#.alias('next_item_prediction')
            , pl.struct(["prev_items", "locale_popular_rec"]).apply(
                        lambda x: get_next_items(x)).alias('next_item_prediction')
        )
)

# eval_res.head().collect()

In [27]:
%%time
model_eval(eval_res)

CPU times: user 4min 33s, sys: 1min 21s, total: 5min 54s
Wall time: 1min 59s


total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
326443,0.000145,0.000493,0.00227


In [18]:
# eval_res.head().collect()

In [28]:
# model_eval(eval_res)

# Save Model

In [32]:
model_dir

'../model_training/popular_v1/'

In [31]:
popular_pl.collect().write_parquet(os.path.join(model_dir, 'popular_rec.parquet'))